In [ ]:
import pandas as pd
import jieba
from snownlp import SnowNLP

In [ ]:
xhs = pd.read_csv('../data/clean/xhs_clean_female.csv')
tmall = pd.read_csv('../data/clean/tmall_clean.csv')

In [ ]:
xhs.head()

In [ ]:
tmall.head()

### Merge Two Platforms

In [ ]:
combine = pd.concat([xhs['text'], tmall['comment']])

df = pd.DataFrame(combine, columns=['text'])

df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

In [ ]:
def topic_angle(keywords, df):
    res = []
    for cell in df['text']:
        for kw in keywords:
            if kw in cell:
                res.append(1)
                break
        else:
            res.append(0)
    return res

In [ ]:
## 应用函数，根据某几列特征的条件，对新特征赋值
def label_get(time_length,broad_rate):
    if time_length<5:
        if broad_rate<=0.51:
            return 0
        else:
            return 1
    elif time_length >=5 and time_length <14:
        if broad_rate<0.91:
            return 0
        else:
            return 1
    else:
        return 0
df['label']=df.apply(lambda x: label_get(x.time,x.broad),axis=1)


In [ ]:
health = ['健康', '营养', '缺乏', '身体', '副作用', #general
         '口腔溃疡', '发育', '头发', '脱发', '肠胃', '眼睛', '骨骼', '指甲', '血管', '护眼', '骨骼', '肚子', '大脑', '生长', '肝脏', '牙齿', '牙龈', '生发', '畸形', '伤口愈合' #身体部位
         '孕吐', '睡眠', '舒服', '免疫力', '便秘', '感冒', '上火', '新陈代谢', '水肿', '失眠', '入睡', '缺钙', '排毒', '睡不着', '疾病', '不适', '生病', '感冒', '疼痛', '难受', '恶心', '过敏', '出血', '亚健康', #病症相关
        '精神', '状态', '疲惫', '疲劳', '能量', '压力', '情绪', '心情', '精力', '元气', '心理'] #心理

beauty = ['美容', '颜值', '气色', '脸色',  #general
        '抗氧化', '皮肤', '美白', '痘痘', '肌肤', '脸上', '保湿', '长痘', '紧致', '护肤', '皱纹', '暗黄', '滋润', '养颜', '淡斑', '肤色', '变白', '细纹', '衰老', '防晒', '色斑', '光泽', '弹性', '祛斑', '祛痘', '补水', '粗糙', '变美', '暗沉', '毛孔', '黑色素', '自由基'#皮肤
         '减肥', '身材', '长胖', '脂肪', '体重', '减脂', '热量'] #体重

benefit = health+beauty

In [ ]:
other_ingredients = ['氨基酸', '蛋白粉', '胶原蛋白', '益生菌', '益生菌', '鱼油', '钙', '铁', '乳酸菌', 'omega',
                   '不饱和脂肪酸', '矿物质', '辅酶', '蔗糖', 'dha', '胡萝卜素', '酵素', '葡萄籽', '蛋白质', '烟酰胺', 
                   '微量元素', '蔓越莓', '叶黄素', '褪黑素', '膳食', '纤维', '活性','肉桂提取物']

level_of_contents = 含量

In [ ]:
traceability= ['成分', '配方', '来源', '营养素', '补充剂', '抗糖', '草本', '黄金',
                    '天然', '添加', '放心', '有机', '自然', '安全', '成分简单', '香精', '防腐剂', '合成', '科技'
                    '澳洲', '日本', '美国', '德国', '进口', '中国', '韩国', '英国']

In [ ]:
flavor = ['口味', '味道', '好吃', '口感', '难吃', '好喝',
         '酸酸', '甜甜', '酸甜', '糖果',
         '水果', '甜橙', '樱桃', '柠檬', '蔓越莓', '橙子', '橘子', '石榴',
        '玫瑰', '针叶',
        '牛油果',
        '无糖',]

In [ ]:
vitamin_format = ['软糖', '胶囊','颗粒', '咀嚼片', '糖果', '泡腾片', '辔腾片', '口服液', '软胶囊',
                  '吞服', '吞咽', '口服', '内服', '外用', '外服', '涂抹', '果肉']

In [ ]:
how_to_use = ['每天', '天天', '两天', '多次', 
              '每天一片', '一天一片', '每天一粒', '每天两粒', '每天一次', '每天一颗', '每天二粒' 
              '早餐', '晚上', '早上', '随餐', '吃饭', '饭后', '睡前', '睡觉', 
              '冬天', '夏天']

In [ ]:
package = ['方便', '携带', '包装', '仔细', '严实', '很大', '颜色', '简单', '盖子', '设计', '好看', '药盒', '可爱',
           '精美', '麻烦', '包装完好', '外观', '小瓶', '大瓶']

In [ ]:
price = ['价格划算','实惠', '便宜', '活动划算', '优惠', '物美价廉', '赠品', '价格便宜', '不贵', '价格比', 
        '物超所值', '赠送', '买一送一', '平价', '物有所值', '性价比超高', '价格优惠', '经济实惠', '价格亲民', 
         '贵', '昂贵']

In [ ]:
life_stages = ['孕期', '孕妈', '怀孕', '孕前', '孕前三个', '怀孕期间', '妊娠', '少女', '准妈妈', 
               '备孕', '哺乳期', '产后', '孕早期', '宝宝健康', '更年期', '青少年']

daily = ['日常补充', '家中常备', '必备', '疫情期间', '换季', '养生', '保养', '保健', '调理', ]

people = ['爸妈', '老公', '家人', '宝宝', '新生儿', '孩子', '儿童', '全家', '父母', '老人']

occassion = ['公司', '办公室', '上班', '加班', '熬夜', '医生建议', '医院', 
             '派对', '喝酒', "油腻", '姨妈', '学生']